# Vision Transformer

In [ ]:
!pip3 install datasets
!pip3 install transformers

In [ ]:
from IPython.display import Image, HTML, display
from matplotlib import pyplot as plt
import numpy as np 
import os
import cv2
import csv

import math
import argparse
import matplotlib
import imghdr
import pickle as pkl
import datetime
from cycler import cycler
from PIL import Image, ImageEnhance

from pathlib import Path
import os.path

from datasets import load_dataset
from datasets import load_metric

from transformers import TrainingArguments
from transformers import ViTFeatureExtractor
from transformers import ViTForImageClassification

import torch

from PIL import Image
import requests
import numpy as np

## Récupération des données



In [ ]:
!rm -rf sample_data
!wget https://cluster.ig.umons.ac.be/HackIA21/databases/FIRE_DATABASE_3.tar
!tar xf FIRE_DATABASE_3.tar
!rm FIRE_DATABASE_3.tar

In [ ]:
# Données de test
!wget http://195.154.53.219/downloads/test.tar
!tar xf test.tar --one-top-level
!rm test.tar

## Dataset 
Récupération des données sous forme de Dataset. Les données "train" sont ensuite splitté en deux afin d'avoir des données de validation. 

Concernant les données de test. On récupère le Dataset prévu à cet effet. 

In [ ]:
ds = load_dataset("imagefolder", data_dir = "./FIRE_DATABASE_3")
ds

Resolving data files:   0%|          | 0/1500 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset imagefolder downloaded and prepared to /root/.cache/huggingface/datasets/imagefolder/default-2d94831408c629a4/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 1500
    })
})

In [ ]:
data = ds['train'].train_test_split(test_size = 0.1)
data

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 1350
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 150
    })
})

Utilisation de dictionnaires afin de relier les ID à l'élément de classification. 

In [ ]:
labels = data['train'].features['label']
labels = data["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

In [ ]:
metric = load_metric('accuracy')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


Chargement du feature extractor. On précise le Transformer que l'on souhaite utiliser. 

In [ ]:
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [ ]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
train_transforms = Compose(
        [
            RandomResizedCrop(feature_extractor.size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

val_transforms = Compose(
        [
            Resize(feature_extractor.size),
            CenterCrop(feature_extractor.size),
            ToTensor(),
            normalize,
        ]
    )

def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

In [ ]:
train_ds = data['train']
val_ds = data['test']
test_ds = load_dataset("imagefolder", data_dir = "./test")

Resolving data files:   0%|          | 0/210 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
test_ds

DatasetDict({
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 210
    })
})

In [ ]:
train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)
test_ds.set_transform(preprocess_val)

In [ ]:
train_ds[0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=300x168 at 0x7FC073EAEF50>,
 'label': 2,
 'pixel_values': tensor([[[ 0.5294,  0.5294,  0.5451,  ...,  0.7255,  0.7176,  0.7176],
          [ 0.5294,  0.5294,  0.5451,  ...,  0.7255,  0.7176,  0.7176],
          [ 0.5373,  0.5373,  0.5451,  ...,  0.7255,  0.7176,  0.7176],
          ...,
          [-0.7490, -0.7490, -0.7490,  ...,  0.0118,  0.0275,  0.0353],
          [-0.7569, -0.7569, -0.7569,  ..., -0.0431, -0.0196, -0.0039],
          [-0.7647, -0.7647, -0.7569,  ..., -0.0667, -0.0431, -0.0275]],
 
         [[ 0.4039,  0.4039,  0.4118,  ...,  0.7412,  0.7333,  0.7333],
          [ 0.4039,  0.4039,  0.4118,  ...,  0.7412,  0.7333,  0.7333],
          [ 0.4118,  0.4118,  0.4118,  ...,  0.7412,  0.7333,  0.7333],
          ...,
          [-0.6706, -0.6706, -0.6706,  ..., -0.2471, -0.2314, -0.2235],
          [-0.6784, -0.6784, -0.6784,  ..., -0.2392, -0.2235, -0.2078],
          [-0.6863, -0.6863, -0.6784,  ..., -0.2392, 

In [ ]:
model_name_or_path = 'google/vit-base-patch16-224-in21k'
model = ViTForImageClassification.from_pretrained(
    model_name_or_path, 
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

Downloading:   0%|          | 0.00/502 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    'finetuned-fire-detection',
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
  hub_strategy="end"
)#

In [ ]:
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

Using cuda_amp half precision backend


In [ ]:
train_results = trainer.train()
# rest is optional but nice to have
torch.save(model.state_dict(), "model.pth")
trainer.save_model('model.h5')
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1350
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 340
  Number of trainable parameters = 85800963


Step,Training Loss,Validation Loss,Accuracy
100,0.175100,0.104411,0.980000
200,0.127200,0.080780,0.980000
300,0.088600,0.078002,0.980000


/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20 bytes but only got 19. Skipping tag 36867
  " Skipping tag %s" % (size, len(data), tag)
***** Running Evaluation *****
  Num examples = 150
  Batch size = 8
Saving model checkpoint to finetuned-fire-detection/checkpoint-100
Configuration saved in finetuned-fire-detection/checkpoint-100/config.json
Model weights saved in finetuned-fire-detection/checkpoint-100/pytorch_model.bin
Feature extractor saved in finetuned-fire-detection/checkpoint-100/preprocessor_config.json
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20 bytes but only got 19. Skipping tag 36867
  " Skipping tag %s" % (size, len(data), tag)
***** Running Evaluation *****
  Num examples = 150
  Batch size = 8
Saving model checkpoint to finetuned-fire-detection/checkpoint-200
Configuration saved in finetuned-fire-detect

***** train metrics *****
  epoch                    =         4.0
  total_flos               = 389721705GF
  train_loss               =      0.1683
  train_runtime            =  0:03:32.19
  train_samples_per_second =      25.448
  train_steps_per_second   =       1.602


In [ ]:
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** Running Evaluation *****
  Num examples = 150
  Batch size = 8


***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =       0.98
  eval_loss               =      0.078
  eval_runtime            = 0:00:05.17
  eval_samples_per_second =     28.996
  eval_steps_per_second   =      3.673


## Évaluation du modèle

In [ ]:
outputs = trainer.predict(test_ds['test'])
print(outputs.metrics)

***** Running Prediction *****
  Num examples = 210
  Batch size = 8


{'test_loss': 0.20394988358020782, 'test_accuracy': 0.9380952380952381, 'test_runtime': 11.3617, 'test_samples_per_second': 18.483, 'test_steps_per_second': 2.376}
